In [30]:
from collections import deque

from parse import parse
from pathlib import Path

import pandas as pd

summary_df_rows = deque()
samples_df_rows = deque()
for txt in Path("./").glob("*.txt"):
    name_parse = parse("results_{mean_ms:d}_{jitter_ms:d}_{correl_pcnt:d}.txt", txt.name)
    mean_latency = 2 * name_parse["mean_ms"] / 1000.0
    jitter = 2 * name_parse["jitter_ms"] / 1000.0
    rho = name_parse["correl_pcnt"] / 100.0
    
    row = {
        "target_mean_latency": mean_latency,
        "jitter": jitter,
        "rho": rho,
    }
    
    with txt.open("r") as fp:
        lines = fp.readlines()
        assert(len(lines) == 1004)
        
        for sample_row in lines[1:-3]:
            sample_parse = parse(
                "{_:d} bytes from 127.0.0.1: icmp_seq={seq:d} ttl={_:d} time={rtt:f} ms",
                sample_row.strip()
            )
            samples_df_rows.append({
                "target_mean_latency": mean_latency,
                "jitter": jitter,
                "rho": rho,
                "seq": sample_parse["seq"],
                "rtt": sample_parse["rtt"] / 1000.0,
            })
        
        summary = lines[-1].strip()
        summary_parse = parse(
            "round-trip min/avg/max/stddev = {min_lat:f}/{avg_lat:f}/{max_lat:f}/{std_lat:f} ms",
            summary,
        )
        row.update({
            "actual_mean_latency": summary_parse["avg_lat"] / 1000.0,
            "std_latency": summary_parse["std_lat"] / 1000.0,
            "min_latency": summary_parse["min_lat"] / 1000.0,
            "max_latency": summary_parse["max_lat"] / 1000.0,
        })
    
    summary_df_rows.append(row)
    
# stationary parameters don't seem to change with correlation, awesome
summary_results = pd.DataFrame(summary_df_rows).groupby([
    "target_mean_latency",
    "jitter",
])["std_latency"].mean()
summary_results

target_mean_latency  jitter
0.1                  0.02      0.012517
0.2                  0.04      0.023106
0.4                  0.08      0.045524
0.8                  0.16      0.092714
1.6                  0.32      0.179005
3.2                  0.64      0.377754
Name: std_latency, dtype: float64

In [62]:
import scipy.stats as st

fit_result_rows = deque()
for (mean, jitter), df in pd.DataFrame(samples_df_rows).groupby(["target_mean_latency", "jitter"]):
    fit_result = st.fit(st.pareto, df["rtt"], bounds={"b": (2, 10), "scale": (0.05, 5.0)})
    assert fit_result.success
    fit_result_rows.append({
        "mean": mean,
        "jitter": jitter,
        "shape": fit_result.params.b,
        "scale": fit_result.params.scale
    })
    
pd.DataFrame(fit_result_rows)

,mean,jitter,shape,scale
0,0.1,0.02,4.447719,0.088318
1,0.2,0.04,5.921988,0.177230
2,0.4,0.08,6.931734,0.351279
3,0.8,0.16,7.454323,0.701631
4,1.6,0.32,7.575100,1.394325
5,3.2,0.64,7.536978,2.783698
